# Ask Syracuse Data: Full EDA (All 16 Datasets)

Comprehensive exploratory data analysis of every dataset in the project.  
For each dataset we look at: shape, dtypes, nulls, unique values, distributions, and what queries/analysis are possible.

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 60)
pd.set_option('display.float_format', '{:.2f}'.format)

from pipeline import data_utils
print('Imports ready')

## Helper: Dataset Profile Function

In [ ]:
def profile_dataset(df, name):
    """Print a full profile of a dataset."""
    print(f"{'='*70}")
    print(f"  {name.upper()}")
    print(f"{'='*70}")
    print(f"Shape: {df.shape[0]:,} rows x {df.shape[1]} columns")
    print(f"Memory: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    print()
    
    # Column types
    print("COLUMNS & TYPES:")
    for col in df.columns:
        nulls = df[col].isna().sum()
        null_pct = nulls / len(df) * 100
        nunique = df[col].nunique()
        print(f"  {col:30s} {str(df[col].dtype):15s} {nunique:>6} unique  {nulls:>6} nulls ({null_pct:.1f}%)")
    print()
    
    # Numeric columns summary
    num_cols = df.select_dtypes(include='number').columns.tolist()
    if num_cols:
        print("NUMERIC SUMMARY:")
        display(df[num_cols].describe())
    
    # Categorical columns - top values
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    if cat_cols:
        print("\nTOP VALUES (categorical columns):")
        for col in cat_cols:
            if df[col].nunique() <= 30:
                print(f"\n  {col} ({df[col].nunique()} unique):")
                vc = df[col].value_counts().head(10)
                for val, cnt in vc.items():
                    print(f"    {str(val):40s} {cnt:>6,} ({cnt/len(df)*100:.1f}%)")
            else:
                print(f"\n  {col}: {df[col].nunique()} unique (top 5: {df[col].value_counts().head(5).index.tolist()})")
    
    # Date columns
    date_cols = df.select_dtypes(include=['datetime64', 'datetimetz']).columns.tolist()
    if date_cols:
        print("\nDATE RANGES:")
        for col in date_cols:
            print(f"  {col}: {df[col].min()} to {df[col].max()}")
    
    # Year column if exists
    if 'year' in df.columns:
        print(f"\nYEAR DISTRIBUTION:")
        vc = df['year'].value_counts().sort_index()
        for yr, cnt in vc.items():
            print(f"  {yr}: {cnt:>6,} rows")
    
    print(f"\n{'-'*70}\n")
    return {
        'name': name,
        'rows': len(df),
        'cols': len(df.columns),
        'memory_mb': df.memory_usage(deep=True).sum() / 1024**2,
        'columns': list(df.columns),
        'null_pct': {col: df[col].isna().mean()*100 for col in df.columns},
    }

---
## 1. Code Violations (~138K rows)
**Source:** Department of Neighborhood & Business Development (NBD) - Division of Code Enforcement  
**Description:** All code violations reported to Code Enforcement (2017-present). The Division of Code Enforcement maintains Housing and Property codes and enforces compliance with the Syracuse Zoning Ordinance. When a structure is not in compliance, it can be cited for a violation. The owner has a set time to bring the structure into compliance. Cases remain open until fixed or referred to the law department.

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `x`, `y` | Float | Map projection coordinates |
| `sbl` | String | Section, Block, Lot -- property tax map number unique to this property |
| `complaint_address` | String | Parcel address associated with the Code Enforcement case |
| `complaint_zip` | Int | Parcel zip code associated with the Code Enforcement case |
| `complaint_number` | String | Violation's parent complaint record case number |
| `complaint_type_name` | String | Type of complaint filed |
| `violation_id` | Int | Unique violation identifier |
| `open_date` | Date | Violation's parent complaint record open date |
| `violation_number` | String | Violation identifier string |
| `violation` | String | The violation type -- references the specific code in violation, including the codebook name and section |
| `violation_date` | Date | Date the violation was cited |
| `comply_by_date` | Date | Date by which the owner must comply and address the issued code violation |
| `status_type_name` | String | Status of the violation as of the date of extract (Open, Closed) |
| `status_date` | Date | Date the status was changed; if "Closed," reflects the date the violation was closed. ~88.6% null |
| `issued_to` | String | Name of contact the violation was issued to (default: assessed owner at time of citation) |
| `issued_to_address` | String | Street address of contact violation was issued to |
| `issued_to_city` | String | City of contact violation was issued to |
| `issued_to_zip` | String | Zip code of contact violation was issued to |
| `inspector_id` | Int | Code Enforcement inspector identifier |
| `neighborhood` | String | Neighborhood where the complaint address is located |
| `vacant` | String | Is the parcel a Code Enforcement-monitored vacant structure? "Residential" or "Commercial" = Yes; Null = No. ~86.2% null |
| `latitude` | Float | Latitude of the parcel |
| `longitude` | Float | Longitude of the parcel |
| `objectid` | Int | ArcGIS system field |

**Computed columns (added by pipeline):** `days_to_comply`, `days_open`, `cert_duration_days`

In [ ]:
violations = data_utils.load_code_violations()
prof_violations = profile_dataset(violations, 'Code Violations')
violations.head(3)

In [ ]:
# Violations deep-dive: status distribution, time trends, geographic spread
print("Status breakdown:")
print(violations['status_type_name'].value_counts())
print(f"\nNeighborhoods with most violations (top 10):")
print(violations['neighborhood'].value_counts().head(10))
print(f"\nZIP codes:")
if 'complaint_zip' in violations.columns:
    print(violations['complaint_zip'].value_counts().head(10))
print(f"\nDate range: {violations.get('violation_date', pd.Series()).min()} to {violations.get('violation_date', pd.Series()).max()}")

---
## 2. Crime Data (~32.8K rows)
**Source:** Syracuse Police Department (SPD)  
**Description:** Calls for service responded to by SPD (2022-2025). Excludes rape offenses and sealed cases. Addresses anonymized to the 100-block level for privacy. Crimes reported under FBI Uniform Crime Reports (UCR): **Part 1** (homicide, robbery, aggravated assault, burglary, larceny-theft, motor vehicle theft) and **Part 2** (all other offenses, quality-of-life). The date/time provided is when the crime was reported, not necessarily when it occurred. 2025 data is partial (33 rows).

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `dateend` | Datetime | Date the crime was **reported** (may have occurred earlier). Mixed formats across source CSVs |
| `timestart` | Int/String | Start of time window in military time (2400 format). Burglaries/larcenies are often a range |
| `timeend` | Int | End of time window in military time (2400 format) |
| `address` | String | Where the crime occurred, anonymized to the 100-block level for resident privacy |
| `code_defined` | String | The offense type / crime classification (e.g., "Burglary", "Larceny-Theft", "Aggravated Assault") |
| `arrest` | String | Whether an arrest was made ("Yes" or "No"). Nulls filled with "No" by pipeline |
| `larcenycode` | String | Larceny sub-classification (e.g., "All Other", "From Building", "Shoplifting"). Only for larceny offenses; ~53.7% null |
| `qualityoflife` | String | Quality-of-life offense flag. Only in 2023+ Part 2 data; ~59.2% null |
| `objectid` | Int | ArcGIS system field |
| `zip` | Int (nullable) | ZIP code derived from lat/long using nearest Syracuse ZIP centroid (computed by pipeline) |
| `neighborhood` | String | Neighborhood derived from geocoding (normalized by pipeline) |
| `latitude` | Float | Latitude coordinate (approximate, block level) |
| `longitude` | Float | Longitude coordinate (approximate, block level) |
| `year` | Int | Year extracted from `dateend` |
| `crime_part` | Int | 1 = Part 1 (serious) offense, 2 = Part 2 (other) offense |
| `x`, `y` | Float | Map projection coordinates (from geocoding, primarily 2022 data) |

In [ ]:
crime = data_utils.load_crime()
prof_crime = profile_dataset(crime, 'Crime')
crime.head(3)

In [ ]:
# Crime deep-dive
print("Crime Part distribution:")
print(crime['crime_part'].value_counts())
print(f"\nTop crime types:")
print(crime['code_defined'].value_counts().head(15))
print(f"\nCrimes by year:")
print(crime['year'].value_counts().sort_index())
print(f"\nNeighborhoods with most crime (top 10):")
print(crime['neighborhood'].value_counts().head(10))
print(f"\nArrest rate:")
print(crime['arrest'].value_counts())

---
## 3. Rental Registry (~13K rows)
**Source:** Division of Code Enforcement (DOCE) - Periodic Inspection Program  
**Description:** Owners of one- or two-family non-owner-occupied dwellings rented or leased within Syracuse must obtain a Rental Registry Certificate every 3 years. This dataset tracks the inspection and certification status of registered rental properties.

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `x`, `y` | Float | Map projection coordinates |
| `sbl` | String | Section, Block, Lot -- property tax number unique to this property |
| `propertyaddress` | String | Parcel street number and street address |
| `zip` | Int | Zip code of the parcel's street address |
| `needsrr` | String | Whether the parcel is flagged as "Rental Registry Required" (1-2 family non-owner occupied) |
| `inspect_period` | Date | Date associated with the RR case (could be case creation, upcoming inspection, or reapplication due date) |
| `completion_type_name` | String | "Rental Registry Card Issued" or "Family Based Exemption Granted" |
| `completion_date` | Date | Date the completion type was entered on a RR case |
| `valid_until` | Date | Date the current RR certificate expires (3 years from last completion) |
| `rrisvalid` | String | Whether the Rental Registry certificate is currently valid |
| `rr_app_received` | Date | Rental Registry application received date |
| `rr_ext_insp_pass` | Date | Exterior inspection pass date. ~56% null |
| `rr_ext_insp_fail` | Date | Exterior inspection fail date. ~94.6% null |
| `rr_int_insp_fail` | Date | Interior inspection fail date. ~92.2% null |
| `rr_int_insp_pass` | Date | Interior inspection pass date. ~59.1% null |
| `rr_contact_name` | String | Contact name for the rental property |
| `pc_owner` | String | Property card owner name |
| `latitude` | Float | Latitude of the parcel |
| `longitude` | Float | Longitude of the parcel |
| `objectid` | Int | ArcGIS system field |

*Note: `shape` column (100% null GIS artifact) is dropped by the pipeline.*

In [ ]:
rentals = data_utils.load_rental_registry()
prof_rentals = profile_dataset(rentals, 'Rental Registry')
rentals.head(3)

In [ ]:
# Rental deep-dive
print("Completion types:")
print(rentals['completion_type_name'].value_counts())
print(f"\nTop ZIP codes:")
if 'zip' in rentals.columns:
    print(rentals['zip'].value_counts().head(10))
print(f"\nUnique SBLs (properties): {rentals['sbl'].nunique()}")

---
## 4. Vacant Properties (~1.7K rows)
**Source:** Department of Neighborhood & Business Development (NBD)  
**Description:** Houses in Syracuse identified as vacant by NBD. Includes Vacant Property Registry (VPR) status -- whether the owner has registered and maintains the vacant property.

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `x`, `y` | Float | Map projection coordinates |
| `sbl` | String | Section, Block, Lot -- tax map ID unique to this property |
| `propertyaddress` | String | Parcel street address |
| `zip` | Int | Parcel zip code |
| `owner` | String | Assessed owner name |
| `owneraddress` | String | Assessed owner address |
| `vacant` | String | Vacant structure type: "Residential" (~90.2%) or "Commercial" (~9.8%) |
| `neighborhood` | String | Neighborhood the property is in |
| `vpr_result` | String | VPR status: "Terminated", "Valid - Year 1/2/3/4+", or "Exempt at this time" |
| `completion_date` | Date | VPR certificate issuance date. ~77.3% null |
| `completion_type_name` | String | VPR completion type |
| `valid_until` | Date | VPR certificate expiration date |
| `vpr_valid` | String | Does property have a valid VPR certificate? ("Y" or "N") |
| `latitude` | Float | Latitude of the property |
| `longitude` | Float | Longitude of the property |
| `objectid` | Int | ArcGIS system field |

In [ ]:
vacant = data_utils.load_vacant_properties()
prof_vacant = profile_dataset(vacant, 'Vacant Properties')
vacant.head(3)

In [ ]:
# Vacant deep-dive
print("By neighborhood (top 10):")
print(vacant['neighborhood'].value_counts().head(10))
print(f"\nBy ZIP:")
if 'zip' in vacant.columns:
    print(vacant['zip'].value_counts().head(10))
print(f"\nVPR Valid:")
if 'vpr_valid' in vacant.columns:
    print(vacant['vpr_valid'].value_counts())
print(f"\nVPR Result:")
if 'vpr_result' in vacant.columns:
    print(vacant['vpr_result'].value_counts())

---
## 5. Unfit Properties (353 rows)
**Source:** Division of Code Enforcement (DOCE)  
**Description:** Houses declared unfit for habitation by Code Enforcement. Process: community complaint -> inspector inspection -> violation citation -> comply-by date. Non-compliance may lead to Law Department referral, Bureau of Administrative Adjudication, or demolition pipeline.

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `complaint_number` | String | Case number of violation's parent complaint record |
| `address` | String | Street address of the property with the violation |
| `area_involved` | String | Physical area the violation applies to, per inspector |
| `zip` | Int | Zip code of the property |
| `sbl` | String | Section, Block, Lot -- property tax number unique to this property |
| `parcel_id` | Int | Property parcel number in the NYS Property database |
| `violation` | String | Violation type -- the specific code in violation (codebook name and section) |
| `violation_date` | Date | Date the violation was cited (stored as epoch milliseconds) |
| `comply_by_date` | Date | Violation compliance deadline per inspector |
| `status_type_name` | String | Violation status: Open, Closed, or Void |
| `department_name` | String | Department the violation/complaint belongs to |
| `status_date` | Date | Date the violation status was created or changed |
| `complaint_type_name` | String | Violation's parent complaint type |
| `comp_status` | String | Status of the parent complaint |
| `comp_open_date` | Date | Date the parent complaint was opened |
| `comp_close_date` | Date | Date the parent complaint was closed |
| `article` | String | Article of the specific code in violation |
| `corrective_action` | String | Corrective action required for compliance, per inspector (249 unique descriptions) |
| `owner_name` | String | Registered owner name |
| `owner_address` | String | Owner's street address |
| `owner_city` | String | Owner's city |
| `owner_state` | String | Owner's state (abbreviation) |
| `owner_zip_code` | String | Owner's zip code |
| `latitude` | Float | Latitude of the property |
| `longitude` | Float | Longitude of the property |
| `violation_number` | String | Violation identifier |

*Note: `vacant` column (100% null) is dropped by the pipeline.*

In [ ]:
unfit = data_utils.load_unfit_properties()
prof_unfit = profile_dataset(unfit, 'Unfit Properties')
unfit.head(3)

---
## 6. Trash Pickup (~41K rows)
**Source:** City of Syracuse - Department of Public Works / Department of Assessment  
**Description:** Trash and recycling pickup schedule by individual address for 2025. Lists which day of the week trash is collected and which recycling week (A or B) applies.

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `sbl` | String | Section, Block, Lot -- property tax number unique to this property |
| `stnum` | String | Parcel street number |
| `stname` | String | Parcel street name |
| `fulladdres` | String | Full street address (number + name) |
| `zip` | Int | Parcel zip code |
| `sanitation` | String | Day that trash is picked up on a non-holiday week (Monday-Friday) |
| `recyclingw` | String | Which week recycling services are provided: "A" or "B" |

In [ ]:
trash = data_utils.load_trash_pickup()
prof_trash = profile_dataset(trash, 'Trash Pickup')
trash.head(3)

---
## 7. Historical Properties (3,486 rows)
**Source:** Syracuse Landmark Preservation Board  
**Description:** Properties determined to be historic -- locally protected sites and/or National Register of Historic Places eligible. Maintained by Syracuse's Landmark Preservation Board.

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `sbl` | String | Section, Block, Lot -- unique parcel identifier |
| `property_address` | String | Address of the property or historic district (parcel address, not mailing address) |
| `zip` | Int | Zip code of the property or district |
| `lpss` | String | Locally Protected Site Status: "Local Protected Site or Local District", "Eligible/Architecturally Significant", or empty |
| `nr_eligible` | String | National Register eligibility: "NR Listed" (on the registry), "NR Eligible (SHPO)" (eligible via State Historic Preservation Office), or empty (neither) |
| `latitude` | Float | Latitude of the property |
| `longitude` | Float | Longitude of the property |

In [ ]:
historical = data_utils.load_historical_properties()
prof_historical = profile_dataset(historical, 'Historical Properties')
historical.head(3)

In [ ]:
# Historical deep-dive
print("National Register eligible:")
if 'nr_eligible' in historical.columns:
    print(historical['nr_eligible'].value_counts())
print(f"\nLPSS (Local Protected Sites):")
if 'lpss' in historical.columns:
    print(historical['lpss'].value_counts())

---
## 8. Assessment Roll (~41K rows)
**Source:** City of Syracuse - Department of Assessment  
**Description:** Tentative tax roll for 2026. Valuation date: Jan 1, 2026. Uniform Percent of Value: 52.00%. Contains property classifications, assessed values, ownership, and tax exemptions for all parcels in Syracuse.

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `sbl` | String | Section, Block, Lot -- tax map number unique to this property |
| `property_address` | String | Full property address |
| `property_city` | String | City, state, and zip code of the property (ZIP extracted by pipeline via regex) |
| `dimensions` | String | Width and length of the property in feet, plus features (fireplace, garage, etc.) |
| `property_class` | Int | NYS property class code number |
| `prop_class_description` | String | Property class description (e.g., "One Family Year-Round Residence", "Apartment", "Vacant Land") |
| `primary_owner` | String | Primary owner name or LLC/agency name |
| `secondary_owner` | String | Secondary owner name. ~96.3% null |
| `owner_address` | String | Primary owner's address (from Assessment records) |
| `po_box` | String | Owner PO Box. ~96.2% null |
| `school_taxable` | Float | School taxable value |
| `municipality_taxable` | Float | Municipality taxable value |
| `county_taxable` | Float | County taxable value |
| `total_assessment` | Float | Total assessed value of the property (aliased as `total_av` in pipeline schema) |
| `exemption_1_description` thru `exemption_6_description` | String | Tax exemption descriptions. 67-100% null; most properties have 0-1 exemptions |
| `exemption_1_amt` thru `exemption_6_amt` | Float | Tax exemption amounts |

**Pipeline-derived column:** `zip` -- extracted from `property_city` via regex (e.g., "Syracuse, NY 13205" -> 13205)

In [ ]:
assessment = data_utils.load_assessment_roll()
prof_assessment = profile_dataset(assessment, 'Assessment Roll')
assessment.head(3)

In [ ]:
# Assessment deep-dive
print("Property class distribution:")
print(assessment['prop_class_description'].value_counts().head(15))
if 'total_av' in assessment.columns:
    print(f"\nTotal assessed value stats:")
    print(assessment['total_av'].describe())
    print(f"\nAvg assessment by top property classes:")
    top = assessment.groupby('prop_class_description')['total_av'].agg(['mean', 'count']).sort_values('count', ascending=False).head(10)
    top['mean'] = top['mean'].map('${:,.0f}'.format)
    print(top)

---
## 9. SYRCityline Requests (~116K rows)
**Source:** City of Syracuse via SeeClickFix software  
**Description:** Resident service requests (2021-present) made through SYRCityline. Covers trash pickup, housing maintenance, potholes, street lights, water, sewer, and more. Requests can be submitted via web, mobile app, or phone.

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `id` | String | Unique request identifier |
| `summary` | String | User-selected category for the complaint (e.g., "Large or Bulk Items", "Weekly Trash Pickup", "Home & Building Maintenance", "Sewer Back-ups") |
| `rating` | String | Number of followers on the request in SeeClickFix |
| `address` | String | Address of the service request, provided by the community member |
| `description` | String | Write-up of the request/complaint, provided by the community member |
| `agency_name` | String | City department assigned (e.g., "Garbage, Recycling & Graffiti", "Housing & Property Maintenance", "Streets, Sidewalks & Transportation") |
| `request_type` | Float | Numeric request type code |
| `latitude` | Float | Latitude (renamed from `lat` by pipeline) |
| `longitude` | Float | Longitude (renamed from `lng` by pipeline) |
| `created_at_local` | Datetime | When the request was submitted (format: MM/DD/YYYY - HH:MM AM/PM) |
| `acknowledged_at_local` | Datetime | When the request was acknowledged by the city department. ~80.6% null |
| `closed_at_local` | Datetime | When the request was marked closed |
| `minutes_to_acknowledge` | Float | Minutes from creation to acknowledgment. ~80.6% null |
| `minutes_to_close` | Float | Minutes from creation to closure. Median ~22 hrs, max ~2.7 years |
| `assignee_name` | String | City department assigned to the request |
| `category` | String | Request category (e.g., "Potholes", "Street Lights", "Weekly Trash Pickup", "Water-related Concerns") |
| `sla_in_hours` | Float | Service Level Agreement limit in hours -- how long before escalation to department head |
| `report_source` | String | How the request was submitted: "Web-Mobile", "iPhone", "Portal", "Web-Desktop", "Android", etc. |

**Pipeline-derived columns:** `zip` (from lat/long), `year` (from `created_at_local`)

In [ ]:
cityline = data_utils.load_cityline_requests()
prof_cityline = profile_dataset(cityline, 'SYRCityline Requests')
cityline.head(3)

In [ ]:
# Cityline deep-dive
print("Top categories:")
print(cityline['category'].value_counts().head(15))
print(f"\nTop agencies:")
print(cityline['agency_name'].value_counts().head(10))
print(f"\nReport sources:")
if 'report_source' in cityline.columns:
    print(cityline['report_source'].value_counts())
if 'minutes_to_close' in cityline.columns:
    print(f"\nMinutes to close:")
    print(cityline['minutes_to_close'].describe())

---
## 10. Snow Routes (3,685 rows)
**Source:** City of Syracuse; compiled by Syracuse Community Geography (Sept 2017)  
**Description:** Street segments listed as emergency snow routes in the City Charter (Sec. 15-337). During a declared snow emergency, parking is unlawful on these streets. Derived from a 2016 NYS GIS Clearinghouse street segment file. For reference and planning only.

### Column Dictionary (key columns out of ~70 total)

| Column | Type | Description |
|--------|------|-------------|
| `completest` | String | Complete street name |
| `streetname` | String | Core street name (without prefix/suffix directions) |
| `predirecti` | String | Pre-direction (N, S, E, W) |
| `posttype` | String | Street suffix type (St, Ave, Blvd, etc.) |
| `postdirect` | String | Post-direction |
| `leftpostal` | String | Left-side postal/ZIP code (used by pipeline as `zip`) |
| `rightposta` | String | Right-side postal/ZIP code |
| `speed` | Int | Speed limit |
| `oneway` | String | One-way designation |
| `leftfromad` / `lefttoaddr` | String | Left-side address range (from/to) |
| `rightfroma` / `righttoadd` | String | Right-side address range (from/to) |
| `label` | String | Display label for the route |
| `leftcensus` / `rightcensu` | String | Census tract on left/right side |
| `shape_leng` / `shape_stle` | Float | Geometry length |

*Remaining ~55 columns are NYS GIS street segment standard fields (road metadata, jurisdictions, edit timestamps, etc.). No official descriptions provided on the portal.*

In [ ]:
snow = data_utils.load_snow_routes()
prof_snow = profile_dataset(snow, 'Snow Routes')
snow.head(3)

---
## 11. Bike Suitability (868 rows)
**Source:** Syracuse Metropolitan Transportation Council (SMTC); ratings collected by staff in 2019  
**Description:** Road suitability ratings for bicycle travel in Greater Syracuse. Ratings: **Excellent** (slow/low traffic, some separation from vehicles), **Good** (moderate traffic, some separation), **Fair/Urban Acceptable** (moderate traffic, little separation or higher volumes with some separation), **Poor** (heavy/fast traffic, little/no separation, rough conditions). For planning purposes only.

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `name` | String | Name of the road or road segment |
| `bike_suitability_19` | String | Suitability rating: "Excellent: Highly Recommend", "Good: Acceptable", "Fair: Urban Acceptable", or "Poor: Not suitable" |

In [ ]:
bike_suit = data_utils.load_bike_suitability()
prof_bike_suit = profile_dataset(bike_suit, 'Bike Suitability')
bike_suit.head(3)

---
## 12. Bike Infrastructure (59 rows)
**Source:** Syracuse Metropolitan Transportation Council (SMTC)  
**Description:** Overview of bicycle infrastructure in Syracuse as of 2023. Includes designated bike lanes, bikeways, trails, and sharrows. Not exhaustive -- infrastructure may have been installed since the last update. Some trail portions may extend outside city limits.

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `infrastructure_type` | String | Type of bike infrastructure: "Lanes", "Bikeway", "Trails", or "Sharrows" |
| `trail_name` | String | Name of the bike trail (if applicable) |
| `length_mi` | Float | Length of this section of bike infrastructure, in miles |
| `globalid` | String | ArcGIS system field |

In [ ]:
bike_infra = data_utils.load_bike_infrastructure()
prof_bike_infra = profile_dataset(bike_infra, 'Bike Infrastructure')
bike_infra.head(3)

---
## 13. Parking Violations (~197K rows)
**Source:** Parking Violations Bureau  
**Description:** Parking tickets issued in Syracuse since 2023. Largest dataset in the project. Note from the portal: many addresses don't map correctly; table view is more complete for tallies.

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `ticket_number` | String | Ticket number -- listed on the physical parking ticket and used for internal tracking |
| `issued_date` | Date | Date and time the parking ticket was issued (stored as epoch milliseconds) |
| `location` | String | Block where the ticket was given (typically rounded to 100-block level) |
| `description` | String | The parking regulation alleged to have been violated |
| `status` | String | Payment status: "Paid in Full", "Collections", "Adjudication Approved", "Promise to Pay", "Adjudication Denied", "Hearing Appeal Affirmed", "Adjudication Partial", "Paid in Collections", "In Bankruptcy", or "Administrative Review PIF" |
| `amount` | Float | Fine amount in dollars |
| `latitude` | Float | Latitude (renamed from `lat` by pipeline) |
| `longitude` | Float | Longitude (renamed from `long` by pipeline) |

**Pipeline-derived columns:** `zip` (from lat/long), `year` (from `issued_date`)

In [ ]:
parking = data_utils.load_parking_violations()
prof_parking = profile_dataset(parking, 'Parking Violations')
parking.head(3)

In [ ]:
# Parking deep-dive
print("Top violation types:")
print(parking['description'].value_counts().head(15))
print(f"\nStatus:")
print(parking['status'].value_counts())
if 'amount' in parking.columns:
    print(f"\nFine amounts:")
    print(parking['amount'].describe())
    print(f"\nAvg fine by top violation types:")
    avg_fines = parking.groupby('description')['amount'].agg(['mean', 'count']).sort_values('count', ascending=False).head(10)
    avg_fines['mean'] = avg_fines['mean'].map('${:.2f}'.format)
    print(avg_fines)

---
## 14. Permit Requests (~47K rows)
**Source:** Central Permit Office  
**Description:** Building permits filed with the Central Permit Office. *Portal note (11/12/2025): This dataset is not currently updating correctly and does not reflect current permit statistics.*

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `permit_number` | String | Permit number assigned by the Central Permit Office |
| `full_address` | String | Full property address |
| `owner` | String | Property owner name |
| `issue_date` | Date | Date the permit was issued (stored as epoch milliseconds) |
| `permit_type` | String | Type of permit (e.g., "Building Permit", "Electrical", "Plumbing") |
| `description_of_work` | String | Description of the work being done (29,283 unique free-text values) |
| `latitude` | Float | Latitude (renamed from `lat` by pipeline) |
| `longitude` | Float | Longitude (renamed from `long` by pipeline) |

**Pipeline-derived columns:** `zip` (from lat/long), `year` (from `issue_date`)

In [ ]:
permits = data_utils.load_permit_requests()
prof_permits = profile_dataset(permits, 'Permit Requests')
permits.head(3)

In [ ]:
# Permits deep-dive
print("Permit types:")
print(permits['permit_type'].value_counts().head(15))
if 'zip' in permits.columns:
    print(f"\nTop ZIP codes:")
    print(permits['zip'].value_counts().head(10))

---
## 15. Tree Inventory (~55K rows)
**Source:** City of Syracuse Parks & Recreation Forestry staff and Cornell Cooperative Extension's CommuniTree Stewards  
**Description:** Inventory of city-managed trees in Syracuse. Entered by Parks & Recreation Forestry staff and the CommuniTree Stewards program.

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `id` | Float | Internal numerical identifier for the tree site |
| `spp_com` | String | Tree species -- common English name (e.g., "Norway Maple", "Red Oak") |
| `spp_bot` | String | Tree species -- Latin/botanical name |
| `dbh` | Float | Diameter at Breast Height -- standard tree measurement at 4.5 feet above ground, in inches |
| `area` | String | Area/neighborhood of the city the tree is in (uses the city's neighborhoods map). Serves as the `neighborhood` equivalent for this dataset |
| `plantdate` | String | Season and year the tree was planted (if known); may be "N/A" or "To Be Determined" |
| `censustrac` | String | Census tract the tree is in (56 unique tracts) |
| `arpa` | String | Whether the tree was planted with American Rescue Plan Act (ARPA) funds ("Yes" ~55.8%, "No" ~44.2%) |
| `address` | Float | Street number closest to the tree |
| `street` | String | Street the tree is located on |
| `x`, `y` | Float | Map projection coordinates |
| `latitude` | Float | Latitude |
| `longitude` | Float | Longitude |

**Pipeline-derived column:** `zip` (from lat/long)

In [ ]:
trees = data_utils.load_tree_inventory()
prof_trees = profile_dataset(trees, 'Tree Inventory')
trees.head(3)

In [ ]:
# Trees deep-dive
print("NOTE: Tree inventory has NO 'condition' column (schema references it but it doesn't exist)")
print("'area' column = neighborhood equivalent\n")
print(f"Top species:")
if 'spp_com' in trees.columns:
    print(trees['spp_com'].value_counts().head(15))
print(f"\nTrees by area/neighborhood (top 10):")
if 'area' in trees.columns:
    print(trees['area'].value_counts().head(10))
print(f"\nARPA (American Rescue Plan Act) trees:")
if 'arpa' in trees.columns:
    print(trees['arpa'].value_counts())
print(f"\nDiameter (DBH) stats:")
if 'dbh' in trees.columns:
    print(trees['dbh'].describe())
print(f"\nCensus tracts: {trees['censustrac'].nunique() if 'censustrac' in trees.columns else 'N/A'}")

---
## 16. Lead Testing (1,185 rows)
**Source:** City of Syracuse (distributed as Excel files, not on ArcGIS portal)  
**Description:** Elevated blood lead levels in children by census tract (2013-2024). Raw data is wide format (tracts as rows, years as columns) with 3 header rows; pipeline melts to long format. Values may be "Suppressed" for small sample sizes (privacy protection), converted to NaN. No official portal data dictionary exists for this dataset.

### Column Dictionary

| Column | Type | Description |
|--------|------|-------------|
| `census_tract` | String | 2020 Census Tract identifier |
| `year` | Int (nullable) | Year of the lead testing data (2013-2024) |
| `pct_elevated` | Float | Percentage of children tested with elevated blood lead levels. "Suppressed" values converted to NaN |

In [ ]:
lead = data_utils.load_lead_testing()
prof_lead = profile_dataset(lead, 'Lead Testing')
lead.head(3)

In [ ]:
# Lead deep-dive
print("Year distribution:")
print(lead['year'].value_counts().sort_index())
print(f"\nCensus tracts: {lead['census_tract'].nunique()}")
if 'pct_elevated' in lead.columns:
    print(f"\nPercent elevated stats:")
    print(lead['pct_elevated'].describe())
    print(f"\nTracts with highest avg elevated lead:")
    top_lead = lead.groupby('census_tract')['pct_elevated'].mean().sort_values(ascending=False).head(10)
    print(top_lead)

---
# Summary: All Datasets at a Glance

In [ ]:
# Build summary table
all_datasets = {
    'Code Violations': violations,
    'Crime': crime,
    'Rental Registry': rentals,
    'Vacant Properties': vacant,
    'Unfit Properties': unfit,
    'Trash Pickup': trash,
    'Historical Properties': historical,
    'Assessment Roll': assessment,
    'Cityline Requests': cityline,
    'Snow Routes': snow,
    'Bike Suitability': bike_suit,
    'Bike Infrastructure': bike_infra,
    'Parking Violations': parking,
    'Permit Requests': permits,
    'Tree Inventory': trees,
    'Lead Testing': lead,
}

summary = []
for name, df in all_datasets.items():
    num_cols = df.select_dtypes(include='number').columns.tolist()
    date_cols = df.select_dtypes(include=['datetime64', 'datetimetz']).columns.tolist()
    has_geo = any(c in df.columns for c in ['latitude', 'longitude', 'lat', 'long', 'x', 'y'])
    has_zip = 'zip' in df.columns or 'complaint_zip' in df.columns
    has_neighborhood = 'neighborhood' in df.columns
    null_pct = df.isna().mean().mean() * 100
    
    summary.append({
        'Dataset': name,
        'Rows': f"{len(df):,}",
        'Cols': len(df.columns),
        'Memory (MB)': f"{df.memory_usage(deep=True).sum()/1024**2:.1f}",
        'Numeric Cols': len(num_cols),
        'Date Cols': len(date_cols),
        'Has Geo': '✓' if has_geo else '',
        'Has ZIP': '✓' if has_zip else '',
        'Has Neighborhood': '✓' if has_neighborhood else '',
        'Avg Null %': f"{null_pct:.1f}%",
    })

summary_df = pd.DataFrame(summary)
print("ALL 16 DATASETS SUMMARY")
print("=" * 100)
display(summary_df)

total_rows = sum(len(df) for df in all_datasets.values())
total_mem = sum(df.memory_usage(deep=True).sum() for df in all_datasets.values()) / 1024**2
print(f"\nTotal: {total_rows:,} rows across 16 datasets, {total_mem:.1f} MB in memory")

---
# Cross-Dataset Join Potential
Which datasets can be linked together and how?

In [ ]:
# Check join keys across datasets
print("JOIN KEY ANALYSIS")
print("=" * 70)

# SBL joins (property-level)
sbl_datasets = {name: df for name, df in all_datasets.items() if 'sbl' in df.columns}
print(f"\nDatasets with SBL (property ID), enables property-level joins:")
for name, df in sbl_datasets.items():
    print(f"  {name}: {df['sbl'].nunique():,} unique SBLs")

# ZIP joins
zip_datasets = {}
for name, df in all_datasets.items():
    if 'zip' in df.columns:
        zip_datasets[name] = df['zip']
    elif 'complaint_zip' in df.columns:
        zip_datasets[name] = df['complaint_zip']
print(f"\nDatasets with ZIP code, enables area-level joins:")
for name, series in zip_datasets.items():
    print(f"  {name}: {series.nunique()} unique ZIPs")

# Neighborhood joins
nbhd_datasets = {name: df for name, df in all_datasets.items() if 'neighborhood' in df.columns}
print(f"\nDatasets with Neighborhood, enables neighborhood-level joins:")
for name, df in nbhd_datasets.items():
    print(f"  {name}: {df['neighborhood'].nunique()} unique neighborhoods")

# Check neighborhood overlap
if len(nbhd_datasets) >= 2:
    names = list(nbhd_datasets.keys())
    print(f"\nNeighborhood overlap between {names[0]} and {names[1]}:")
    set1 = set(nbhd_datasets[names[0]]['neighborhood'].dropna().unique())
    set2 = set(nbhd_datasets[names[1]]['neighborhood'].dropna().unique())
    overlap = set1 & set2
    print(f"  {names[0]}: {len(set1)} | {names[1]}: {len(set2)} | Overlap: {len(overlap)}")
    only1 = set1 - set2
    only2 = set2 - set1
    if only1:
        print(f"  Only in {names[0]}: {sorted(only1)[:10]}")
    if only2:
        print(f"  Only in {names[1]}: {sorted(only2)[:10]}")

---
# Data Quality Check

In [ ]:
# Comprehensive data quality check across all datasets
print("DATA QUALITY REPORT")
print("=" * 70)

quality_issues = []

for name, df in all_datasets.items():
    # Check for duplicate rows
    dupes = df.duplicated().sum()
    if dupes > 0:
        quality_issues.append(f"  {name}: {dupes:,} duplicate rows ({dupes/len(df)*100:.1f}%)")
    
    # Check for high-null columns (>50%)
    for col in df.columns:
        null_pct = df[col].isna().mean() * 100
        if null_pct > 50:
            quality_issues.append(f"  {name}.{col}: {null_pct:.0f}% null")

if quality_issues:
    print("\nIssues found:")
    for issue in quality_issues:
        print(issue)
else:
    print("  No major quality issues found!")

# Check for common column name inconsistencies
print("\nCOLUMN NAME CONSISTENCY:")
all_cols = set()
for name, df in all_datasets.items():
    all_cols.update(df.columns)
# Check for near-duplicates
col_list = sorted(all_cols)
print(f"  Total unique columns across all datasets: {len(col_list)}")
print(f"  Geographic columns found: {[c for c in col_list if c in ['zip', 'complaint_zip', 'latitude', 'longitude', 'lat', 'long', 'x', 'y', 'neighborhood', 'census_tract']]}")

---
# What Can We Do With This Data?

## Currently Supported Queries
- Counts, averages, min/max/sum by group
- Temporal breakdowns (year, month, quarter)
- Cross-dataset joins (violations + rentals, crime + vacant, etc.)
- Rankings, percentiles via LLM SQL path

## New Analysis Opportunities

In [ ]:
# Identify new analysis opportunities based on the data

print("ANALYSIS OPPORTUNITIES")
print("=" * 70)

print("""
1. HOUSING HEALTH INDEX
   Combine: violations + vacant + unfit + rental_registry + assessment
   By: neighborhood or ZIP
   Metric: composite score of property distress
   Data available: All 5 datasets have ZIP or neighborhood

2. CRIME + PROPERTY CONDITIONS CORRELATION
   Combine: crime + violations + vacant_properties
   Question: "Do neighborhoods with more vacant properties have more crime?"
   Data available: All have neighborhood column

3. CITY RESPONSIVENESS ANALYSIS
   Dataset: cityline_requests (minutes_to_close)
   Questions: Which categories get resolved fastest? Which ZIPs wait longest?
   Data available: 116K requests with response times

4. LEAD EXPOSURE + HOUSING CONDITIONS
   Combine: lead_testing + violations (would need census tract to neighborhood/ZIP mapping)
   Question: "Do areas with more code violations have higher lead levels?"
   Gap: lead_testing uses census_tract, others use ZIP/neighborhood

5. TEMPORAL TRENDS (Multi-Year)
   Datasets with year: crime (2022-2025), violations (2017+), cityline, permits, parking
   Questions: "Is crime going up or down?" "Violation trends over time?"
   Data available: Yes

6. TREE CANOPY + NEIGHBORHOOD QUALITY
   Combine: tree_inventory + violations + crime
   Question: "Do neighborhoods with more trees have fewer violations?"
   Data available: All have neighborhood

7. PROPERTY VALUE ANALYSIS
   Dataset: assessment_roll (total_av by prop_class, ZIP)
   Combined with: violations, vacant to see if distressed areas have lower assessments
   Data available: Yes

8. GEOGRAPHIC HOTSPOT MAPPING
   Datasets with lat/long: crime, cityline, parking_violations
   Opportunity: heatmaps, clustering, density analysis
   Currently: only bubble maps and point maps
""")

---
# Column Inventory (Every Column in Every Dataset)

In [ ]:
# Full column inventory for reference
print("FULL COLUMN INVENTORY")
print("=" * 70)
for name, df in all_datasets.items():
    print(f"\n{name} ({len(df):,} rows):")
    for col in df.columns:
        dtype = str(df[col].dtype)
        nunique = df[col].nunique()
        sample = str(df[col].dropna().iloc[0])[:50] if df[col].notna().any() else 'ALL NULL'
        print(f"  {col:35s} {dtype:15s} {nunique:>6} unique  sample: {sample}")

---
# EDA Findings & Inconsistencies

## Dataset Overview (691,827 total rows, 387.6 MB in memory)

| Dataset | Rows | Cols | Geo | ZIP | Neighborhood | SBL | Year |
|---|---:|---:|:---:|:---:|:---:|:---:|:---:|
| Code Violations | 137,663 | 25 | Y | Y | Y | Y | |
| Crime | 32,840 | 17 | Y | Y | Y | | Y |
| Rental Registry | 11,085 | 22 | Y | Y | | Y | |
| Vacant Properties | 1,651 | 17 | Y | Y | Y | Y | |
| Unfit Properties | 353 | 27 | Y | Y | | Y | |
| Trash Pickup | 41,096 | 7 | | Y | | Y | |
| Historical Properties | 3,486 | 7 | Y | Y | | Y | |
| Assessment Roll | 41,367 | 27 | | Y | | Y | |
| Cityline Requests | 116,143 | 19 | Y | Y | | | Y |
| Snow Routes | 3,685 | 70 | | Y | | | |
| Bike Suitability | 868 | 2 | | | | | |
| Bike Infrastructure | 59 | 4 | | | | | |
| Parking Violations | 196,768 | 9 | Y | Y | | | Y |
| Permit Requests | 47,902 | 9 | Y | Y | | | Y |
| Tree Inventory | 55,676 | 15 | Y | Y | | | |
| Lead Testing | 1,185 | 3 | | | | | Y |

---

## SBL (Property-Level) Join Overlap

7 datasets share SBL. Strongest overlaps:

| Dataset A | Dataset B | Shared SBLs | A total | B total | Match % (of smaller) |
|---|---|---:|---:|---:|---:|
| Code Violations | Assessment Roll | 17,074 | 17,348 | 41,367 | 98.4% |
| Code Violations | Trash Pickup | 17,192 | 17,348 | 40,865 | 99.1% |
| Code Violations | Rental Registry | 7,991 | 17,348 | 11,048 | 72.3% |
| Rental Registry | Assessment Roll | 11,028 | 11,048 | 41,367 | 99.8% |
| Code Violations | Vacant Properties | 1,483 | 17,348 | 1,651 | 89.8% |
| Vacant Properties | Assessment Roll | 1,632 | 1,651 | 41,367 | 98.8% |
| Vacant Properties | Unfit Properties | 101 | 1,651 | 305 | 33.1% |
| Trash Pickup | Assessment Roll | 40,535 | 40,865 | 41,367 | 99.2% |
| Historical Properties | Assessment Roll | 3,439 | 3,471 | 41,367 | 99.1% |

Assessment Roll is the universal backbone -- nearly every property in other datasets matches it.

## Neighborhood Join Overlap

Only 3 datasets have `neighborhood`: Code Violations (34), Crime (34), Vacant Properties (33).

- Violations + Crime: 27/41 neighborhoods match
- Violations + Vacant: 33/34 match
- Crime + Vacant: 27/40 match

Tree Inventory uses `area` instead of `neighborhood` (same concept, 31 areas). Schema already correctly uses `area`.

## ZIP Codes

12 Syracuse ZIPs across 12/16 datasets: 13202, 13203, 13204, 13205, 13206, 13207, 13208, 13210, 13214, 13215, 13219, 13224.

Datasets without ZIP: Bike Suitability, Bike Infrastructure, Lead Testing (uses census_tract).

---

## Data Quality Issues Found

### Duplicate Rows (FIXED)
- **Trash Pickup**: 221 duplicate rows (0.5%) -- FIXED: added `.drop_duplicates()` in loader
- **Cityline Requests**: 835 duplicate rows (0.7%) -- FIXED: added `.drop_duplicates()` in loader
- **Historical Properties**: 15 duplicate rows (0.4%) -- FIXED: added `.drop_duplicates()` in loader
- **Permit Requests**: 1 duplicate row (negligible, not fixed)

### High-Null Columns (>50%)

| Dataset | Column | Null % | Notes |
|---|---|---:|---|
| Code Violations | `status_date` | 88.6% | Only populated for closed violations |
| Code Violations | `vacant` | 86.2% | Only "Residential"/"Commercial" when flagged |
| Crime | `arrest` | 86.9% | Was only "Yes" values -- FIXED: nulls now filled with "No" |
| Crime | `larcenycode` | 53.7% | Only applies to larceny offenses |
| Crime | `qualityoflife` | 59.2% | Only for 2023+ data |
| Rental Registry | `rr_ext_insp_fail` | 94.6% | Only populated on failure |
| Rental Registry | `rr_int_insp_fail` | 92.2% | Only populated on failure |
| Rental Registry | `rr_ext_insp_pass` | 56.0% | Only populated on pass |
| Rental Registry | `rr_int_insp_pass` | 59.1% | Only populated on pass |
| Rental Registry | `shape` | 100.0% | FIXED: dropped in loader (GIS artifact) |
| Vacant Properties | `completion_date` | 77.3% | Only for VPR-certified properties |
| Unfit Properties | `vacant` | 100.0% | FIXED: dropped in loader |
| Assessment Roll | `secondary_owner` | 96.3% | Rare |
| Assessment Roll | `po_box` | 96.2% | Rare |
| Assessment Roll | `exemption_1_*` through `exemption_6_*` | 67-100% | Most properties have 0-1 exemptions |
| Cityline Requests | `acknowledged_at_local` | 80.6% | Most requests never formally acknowledged |
| Cityline Requests | `minutes_to_acknowledge` | 80.6% | Same as above |

### Schema vs Reality Mismatches

| Issue | Status |
|---|---|
| Tree `condition`/`neighborhood` columns | NOT A BUG: schema.py already correctly uses `area` and `dbh`. Only CLAUDE.md was wrong -- FIXED in CLAUDE.md. |
| Unfit `vacant` column is 100% null | FIXED: column dropped in loader |
| Rental Registry `shape` column is 100% null | FIXED: column dropped in loader |
| Crime `arrest` has no "No" values | FIXED: null handling changed from "Unknown" to "No" |
| Code Violations actual rows: 137,663 | FIXED: README updated from "~44K" to "~138K" |
| Cityline/Parking/Permits missing extracted `year` column | FIXED: year extracted from date columns in all 3 loaders |

---

## Unexposed Columns Worth Adding

### High Value (would enable new query types)

| Dataset | Column | Values | Enables |
|---|---|---|---|
| Crime | `larcenycode` | 12 types (All Other, From Building, Shoplifting, etc.) | "What types of larceny are most common?" |
| Crime | `arrest` | Yes (4,315) / No (28,525) | "How many crimes resulted in arrest?" |
| Cityline | `sla_in_hours` | 17 values (0-2605 hrs) | "Which request types have the longest SLA?" |
| Cityline | `summary` | 266 types | More granular than `category` |
| Tree Inventory | `censustrac` | 56 census tracts | Could link to lead testing data |
| Tree Inventory | `arpa` | Yes (55.8%) / No (44.2%) | "How many ARPA-funded trees?" |
| Assessment Roll | `school_taxable` | Varies by exemptions | Tax analysis |

### Medium Value

| Dataset | Column | Notes |
|---|---|---|
| Crime | `qualityoflife` | True/False, but 59% null (only 2023+ data) |
| Vacant Properties | `vacant` | Residential (90.2%) vs Commercial (9.8%) |
| Rental Registry | `rrisvalid` | Yes (40.8%) / No (59.2%) -- current validity |
| Unfit Properties | `corrective_action` | 249 unique text descriptions |
| Parking Violations | `location` | 8,874 unique locations |
| Permit Requests | `description_of_work` | 29,283 unique -- free text, hard to group |

---

## New Cross-Dataset Analysis Opportunities

1. **Housing Health Index** -- Combine violations + vacant + unfit + rental_registry + assessment by neighborhood/ZIP into a composite distress score. All 5 datasets share SBL or ZIP.

2. **Crime vs Property Conditions** -- Do neighborhoods with more vacant properties have more crime? All 3 have neighborhood. 27/41 neighborhoods overlap between crime and violations.

3. **City Responsiveness** -- Cityline has 116K requests with `minutes_to_close`. Which categories resolve fastest? Which ZIPs wait longest? Median close time is 1,327 minutes (~22 hours), max is 1,408,207 minutes (~2.7 years).

4. **Lead Exposure vs Housing** -- Lead testing uses census_tract (157 tracts). Tree inventory also has `censustrac` (56 tracts). Could bridge lead data to property data through census tracts.

5. **Tree Canopy vs Neighborhood Quality** -- Tree inventory `area` column maps to neighborhoods. Compare tree density/diameter against violations and crime rates.

6. **Property Value vs Distress** -- Assessment roll has property values for 98.8% of vacant properties (via SBL). Compare assessed values for vacant vs non-vacant, violation-heavy vs clean.

7. **Temporal Trends** -- Crime (2022-2025), violations (2017+), permits, cityline all have date columns. Year-over-year trend analysis. Note: 2025 crime data is only 33 rows (Jan 1-5).

8. **Geographic Hotspot Mapping** -- 9 datasets have lat/long. Density heatmaps, clustering, hotspot detection. Currently only bubble and point maps in the app.

---

## All Fixes Applied

| # | Fix | What Changed | File(s) Modified |
|---|---|---|---|
| 1 | Pre-merged crime CSV | Created `crime_merged.csv` (32,840 rows) with ZIP, neighborhood, year, crime_part pre-computed. Eliminates geocoding on every load. 1.8x faster. | `data/raw/crime_merged.csv` |
| 2 | `load_crime()` uses merged file | Fast path reads single CSV. Falls back to 6-CSV + geocoding if missing. Also applies null handling to merged path. | `pipeline/data_utils.py` |
| 3 | README violation count | Changed "~44K" to "~138K" (137,663 violation rows). Also fixed vacant from "~1.4K" to "~1.7K". | `README.md` |
| 4 | Trash Pickup duplicates | Added `.drop_duplicates()` in loader. Removes 221 duplicate rows (41,096 -> 40,875). | `pipeline/data_utils.py` |
| 5 | Cityline duplicates | Added `.drop_duplicates()` in loader. Removes 835 duplicate rows (116,143 -> 115,308). | `pipeline/data_utils.py` |
| 6 | Historical Properties duplicates | Added `.drop_duplicates()` in loader. Removes 15 duplicate rows (3,486 -> 3,471). | `pipeline/data_utils.py` |
| 7 | Rental Registry `shape` column | Dropped 100% null column in loader. Saves memory. 22 -> 21 columns. | `pipeline/data_utils.py` |
| 8 | Unfit Properties `vacant` column | Dropped 100% null column in loader. 27 -> 26 columns. | `pipeline/data_utils.py` |
| 9 | Crime `arrest` null values | Changed null handling label from "Unknown" to "No". Now shows Yes (4,315) / No (28,525) instead of Yes/null. Regenerated crime_merged.csv. | `pipeline/data_utils.py`, `data/raw/crime_merged.csv` |
| 10 | Year extraction for 3 datasets | Added `year = dt.year` extraction in loaders for Cityline (from `created_at_local`, 2021-2025), Parking Violations (from `issued_date`, 2023-2025), and Permit Requests (from `issue_date`, 1980-2025). Schema already had `year` in `temporal_group_map` but the column didn't exist at load time. | `pipeline/data_utils.py` |
| 11 | CLAUDE.md tree_inventory entry | Fixed "neighborhood, condition" to "area, spp_com, zip; avg/min/max dbh" to match actual schema and data. | `CLAUDE.md` |

All 13 benchmark tests pass after fixes (100%).